<a href="https://colab.research.google.com/github/Tanishk2610/AISD/blob/main/Zero_shot_Image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from transformers import AutoProcessor
import torch

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import os
from PIL import Image

In [ ]:
image_directory = "/content/drive/MyDrive/84c3700e99c9bde25a14514d415fa349"

In [ ]:
image_paths = [os.path.join(image_directory, fname) for fname in os.listdir(image_directory) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
labels = ["Informational", "Hateful", "Memes"]

In [ ]:
images = [Image.open(image_path) for image_path in image_paths]

In [ ]:
results = {}
category_images = {"Informational": [], "Hateful": [], "Memes": []}
total_images = len(image_paths)

for idx, image_path in enumerate(image_paths, start=1):
    try:
        image = Image.open(image_path)

        inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)

        with torch.no_grad():
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)

        image_results = {label: prob.item() for label, prob in zip(labels, probs[0])}
        results[image_path] = image_results

        max_label = max(image_results, key=image_results.get)
        category_images[max_label].append(image_path)

        print(f"\nImage {idx}/{total_images}: {os.path.basename(image_path)}")
        print("-" * 30)
        for label, prob in sorted(image_results.items(), key=lambda item: item[1], reverse=True):
            print(f"  {label:15}: {prob:.2%}")

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

print("\nSummary of results:")
print("=" * 30)

for category, image_list in category_images.items():
    print(f"\n{category} Images ({len(image_list)}):")
    print("-" * 30)
    for image_path in image_list:
        print(f"{os.path.basename(image_path)}")


Image 1/151: 3375147572933695231_114.png
------------------------------
  Memes          : 97.68%
  Informational  : 1.44%
  Hateful        : 0.88%

Image 2/151: 3375147572933695231_111.png
------------------------------
  Memes          : 65.03%
  Informational  : 30.67%
  Hateful        : 4.30%

Image 3/151: 3368226813963468457_17.png
------------------------------
  Memes          : 67.79%
  Informational  : 19.62%
  Hateful        : 12.59%

Image 4/151: 3370005810858921841_143.png
------------------------------
  Informational  : 95.80%
  Memes          : 2.93%
  Hateful        : 1.28%

Image 5/151: 3364931932520410691_20.png
------------------------------
  Informational  : 78.15%
  Memes          : 17.69%
  Hateful        : 4.16%

Image 6/151: 3364634843919883159_62.png
------------------------------
  Memes          : 59.45%
  Hateful        : 28.64%
  Informational  : 11.91%

Image 7/151: 3367208155657066569_31.png
------------------------------
  Hateful        : 89.51%
  Mem